In [ ]:
%load_ext autoreload
%autoreload 2
# Add parent directory into system path
import sys, os
sys.path.insert(1, os.path.abspath(os.path.normpath('..')))

import numpy as np
from sdf import *
from scipy.stats import describe

@sdf3
def gyroid(w = 3.14159, t=0):
    def f(p):
        q = w*p
        x, y, z = (q[:, i] for i in range(3))
        return (np.cos(x)*np.sin(y) + np.cos(y)*np.sin(z) + np.cos(z)*np.sin(x) - t)
    return f

In [ ]:
from sdf.mesh import _estimate_bounds, _cartesian_product

# Generate STL from implicit f
output_stl = 'tmp.stl'
f = box(1) & gyroid(w=12)
f.save(output_stl, step=0.01)

from utils import igl
print(os.path.exists(output_stl))
_v, _f = igl.read_triangle_mesh(output_stl)
bv, bf = igl.bounding_box(_v)

# calcualte lvl_set of f
nums = 100
offset = .1
#bounds = _estimate_bounds(f)
bounds = (bv[0], bv[-1]) if np.mean(bv[0]) < np.mean(bv[-1]) else (bv[-1], bv[0])
(x0, y0, z0), (x1, y1, z1) = bounds

X = np.linspace(x0-offset, x1+offset, nums, dtype=np.float32)
Y = np.linspace(y0-offset, y1+offset, nums, dtype=np.float32)
Z = np.linspace(z0-offset, z1+offset, nums, dtype=np.float32)

P = _cartesian_product(X, Y, Z).astype(np.float32)
lvl_set = f(P).astype(np.float32).squeeze()

In [30]:
from utils.geometry import ImportanceImplicitSampler
sampler = ImportanceImplicitSampler(lvl_set)